# PDFを要約

https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_indexes

## 初期設定

In [ ]:
%pip install python-dotenv
%pip install langchain
%pip install openai
%pip install chromadb
%pip install tiktoken
%pip install pypdf

In [1]:
from dotenv import load_dotenv
import os

# secretenvファイルから環境変数をロード
secret_path = os.path.expanduser('~/dotsecret')
load_dotenv(dotenv_path=secret_path)

openai_api_key = os.getenv('OPENAI_API_KEY')

## LangChainの基本

In [5]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat_model = ChatOpenAI()

llm.predict("hi!")

chat_model.predict("hi!")

'Hello! How can I help you today?'

In [7]:
text = "What would be a good company name for a company that makes colorful socks? in japanese"

#llm.predict(text)
# >> Feetful of Fun

chat_model.predict(text)
# >> Socks O'Color

'A good company name for a company that makes colorful socks in Japanese could be "Iroiro Socks" (いろいろソックス). "Iroiro" means "various" or "many colors" in Japanese, which perfectly represents the colorful nature of the socks.'

In [8]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="Japanese", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to Japanese.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("colors")
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['red', 'blue', 'green', 'yellow', 'purple']

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo")
chat([
	SystemMessage(content="日本語で回答して。"),
	HumanMessage(content="ITエンジニアについて30文字で教えて。"),
])

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


AIMessage(content='コンピュータ技術を活用してシステム開発やネットワーク構築を行う技術者。', additional_kwargs={}, example=False)

## テキストを埋め込む

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader

long_text = """
GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。

自然言語処理(NLP)という技術を使い、文章の生成や理解を行うことができます。

これにより、人間と同じような文章を作成することが可能です。

GPT-4は、トランスフォーマーアーキテクチャに基づいており、より強力な性能を発揮します。

GPT-4は、インターネット上の大量のテキストデータを学習し、豊富な知識を持っています。

しかし、2021年9月までの情報しか持っていません。

このモデルは、質問応答や文章生成、文章要約など、様々なタスクで使用できます。

ただし、GPT-4は完璧ではありません。

時々、誤った情報や不適切な内容を生成することがあります。

使用者は、その限界を理解し、

適切な方法で利用することが重要です。
"""
print(len(long_text))
with open("./datasource/long_text.txt", "w") as f:
    f.write(long_text)
    f.close()

loader = TextLoader('./datasource/long_text.txt')

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = len,
)

index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, # Default
    embedding=OpenAIEmbeddings(), # Default
    text_splitter=text_splitter,
).from_loaders([loader])



query = "Q1. インターネット上の何のデータを使って、学習しているの？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

answer_with_sources = index.query_with_sources(query)
print(answer_with_sources)

query = "Q2. GPT4は第何世代のモデル？"
print(f"\n\n{query}")
answer = index.query(query)
print(answer)

answer_with_sources = index.query_with_sources(query)
print(answer_with_sources)

368


Q1. インターネット上の何のデータを使って、学習しているの？
 GPT-4は、インターネット上の大量のテキストデータを学習しています。
{'question': 'Q1. インターネット上の何のデータを使って、学習しているの？', 'answer': ' GPT-4 is trained using a large amount of text data from the internet.\n', 'sources': './long_text.txt'}


Q2. GPT4は第何世代のモデル？
 GPT-4は、OpenAIが開発したAI技術であるGPTシリーズの第4世代目のモデルです。
{'question': 'Q2. GPT4は第何世代のモデル？', 'answer': ' GPT-4 is the fourth generation model of the GPT series developed by OpenAI.\n', 'sources': './long_text.txt'}


## PDFを学習させたChatGPTの実装方法

[PDFを学習させたChatGPTの実装方法【Python / LangChain / FAQチャットボット】](https://zenn.dev/umi_mori/articles/langchain-indexes-pdf)

In [19]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator

loader = PyPDFLoader("https://blog.freelance-jp.org/wp-content/uploads/2023/03/FreelanceSurvey2023.pdf")

index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, # Default
    embedding=OpenAIEmbeddings(), # Default
).from_loaders([loader])

query = "フリーランス白書のPDFにおける、今回の調査の有効回答数は？"

answer = index.query(query)
print(answer)

 878名(内フリーランス・パラレルキャリア活動者 850名)


## PDFからindexを生成してディレクトリに永続化する

In [24]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator

# データの追加
loader_sre = PyPDFLoader(os.path.expanduser('./the-site-reliability-workbook-next18.pdf'))

index_sre = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=OpenAIEmbeddings(),
    vectorstore_kwargs={"persist_directory": "datastore"}, # datastoreフォルダにvectorstoreを保存する
).from_loaders([loader_sre])

# 検索
query_sre = "site-reliabilityのために一番最初にするべきことは？日本語で答えてください"
answer_sre = index_sre.query(query_sre)
print(answer_sre)


 リライアビリティが最も重要な機能であることを理解し、それを実現するために、ソフトウェアと人間が協力して協調しなければならないことを理解することです。


In [25]:
index_sre.query("避けるべきことはなんですか？日本語でお願いします")

' インタラプトを受ける時間を予定することを避けるべきです。ケーススタディ2では、リモートチームがオンコールを減らしたとき、チームメンバーはプロジェクトに集中する時間を得ることができました。'

In [20]:
index.query("PDFの著者は？")

' 一般社団法人プロフェッショナル＆パラレルキャリアフリーランス協会'

## LangChainでindex化されたPDFをデータストアから読み込む

In [28]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore = Chroma(persist_directory="datastore", embedding_function=OpenAIEmbeddings())
index_from_store = VectorStoreIndexWrapper(vectorstore=vectorstore)

# 検索
query = "site-reliabilityのために一番最初にするべきことは？日本語で答えてください"
answer = index_from_store.query(query)
print(answer)

 リライアビリティが最も重要な機能であることを理解し、それを実現するために、ソフトウェアと人間が協力して協調しなければならないことを理解することです。


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.chat_models import ChatOpenAI

vectorstore = Chroma(persist_directory="datastore", embedding_function=OpenAIEmbeddings())
index_from_store = VectorStoreIndexWrapper(vectorstore=vectorstore)

# 検索
query = "site-reliabilityのために一番最初にするべきことは？日本語で答えてください"
answer = index_from_store.query(query, llm=ChatOpenAI(model_name="gpt-4"))
print(answer)

In [33]:
index_from_store.query("本書の要点を教えてください")

' 第1冊のSRE本と合わせて読むことをお勧めします。本書では、GoogleのSREが実践している方法を中心に、The Home DepotやThe New York Timesなどの伝統的な企業からデジタルスタートアップまで、他の企業からの視点も含めて扱っています。'